In [1]:
import voxel_slam

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
import numpy as np
import mrob 
import open3d as o3d 

import pathlib
import copy
import os 

### I/O 

In [3]:
def read_depth_to_rgb_map(association_path):
    depth_to_rgb_map = dict()
    with open(association_path) as data:
        for line in data:
            _, color_name, _, depth_name = line.split()
            color_name = color_name.strip('\n').lstrip("color/")
            depth_name = depth_name.strip('\n').lstrip("depth/")
            depth_to_rgb_map[depth_name] = color_name
    return depth_to_rgb_map

def parse_trajectories(input_path):
    ts = dict()
    with open(input_path) as data:
        for line in data:
            line_tokens = line.strip('\n').split()
            
            timestamp = line_tokens[0].replace('.', '')
            timestamp = "0" * (12 - len(timestamp)) + timestamp
            
            trajectory = np.asarray(list(map(float, line_tokens[1:])))
            ts.update({timestamp: trajectory})
    return ts


def filename_to_pose(depth_filename: str, depth_to_rgb_map: str, poses: dict):
    rgb_filename = depth_to_rgb_map[depth_filename].rstrip(".png")
    return poses["0" * (12 - len(rgb_filename)) + rgb_filename]


def trajectory_to_se3(trajectory):
    t, Q = trajectory[:3], trajectory[3:]
    R = mrob.geometry.SO3(mrob.geometry.quat_to_so3(Q))
    return mrob.geometry.SE3(R, t)

def parse_trajectories_float(input_path, ts_multiplier=1e9):
    ts = dict()
    with open(input_path) as data:
        for line in data:
            if line.startswith('#'):
                continue
            line_tokens = line.strip('\n').split()
            
            timestamp = float(line_tokens[0]) * ts_multiplier
            
            trajectory = np.asarray(list(map(float, line_tokens[1:])))
            ts.update({timestamp: trajectory})
    return ts

In [4]:
def read_hilti_sequence(ts_to_quat, ts_to_depth_path, start_of_sequence=0, number_of_clouds=-1, center_distance_threshold=1.5):
    poses = []
    clouds = []

    lidar_so3 = mrob.geometry.SO3(mrob.geometry.quat_to_so3(np.asarray([ 0.7071068, -0.7071068, 0, 0 ])))
    lidar_t = np.asarray([ -0.001, -0.00855, 0.055 ])   
    imu_to_lidar_se3 = mrob.geometry.SE3(lidar_so3, lidar_t).T()

    for ts in sorted(ts_to_quat)[start_of_sequence : start_of_sequence + number_of_clouds]:
        imu_pose = trajectory_to_se3(ts_to_quat[ts]).T()

        pose = imu_pose @ imu_to_lidar_se3
        cloud = o3d.io.read_point_cloud(str(ts_to_depth_path[ts]))
        cloud_points = np.asarray(cloud.points)
        f = np.where(np.linalg.norm(cloud_points, axis=1) > center_distance_threshold)[0] 

        cloud.points = o3d.utility.Vector3dVector(cloud_points[f])

        cloud.paint_uniform_color([0.0, 0.0, 0.0])
        
        poses.append(pose)
        clouds.append(cloud)

    return clouds, poses

In [5]:
def read_vector_sequence(ts_to_quat, ts_to_pcd_path, start_of_sequence=0, number_of_clouds=-1, enable_down_sampling=True):
    poses = []
    clouds = [] 

    for ts in sorted(ts_to_quat)[start_of_sequence : start_of_sequence + number_of_clouds]:
        pose = trajectory_to_se3(ts_to_quat[ts]).T()
        
        # pcd_name = f"{ts}.pcd"
        pcd = o3d.io.read_point_cloud(str(ts_to_pcd_path[ts]), remove_infinite_points=True)
        pcd.paint_uniform_color([0.0, 0.0, 0.0])

        clouds.append(pcd.uniform_down_sample(2) if enable_down_sampling else pcd)
        poses.append(pose)

    return clouds, poses 

In [9]:
pcd = o3d.io.read_point_cloud("/home/ach/Desktop/datasets/hilti/out2/1649764528.063481000.pcd")

o3d.io.write_point_cloud("pcd_ascii.pcd", pcd,  write_ascii=True)

True

## Clustering optimization

In [21]:
def get_optimized_poses(clouds_cp, 
                        poses_cp, 
                        voxel_size=2, 
                        ransac_distance_threshold=0.005, 
                        filter_function=lambda points: True,
                        enable_filter_voxels=True,
                        filter_cosine_distance_threshold=0.2, 
                        filter_preserve_non_informative_voxels=False):
    clouds = copy.deepcopy(clouds_cp)
    poses = copy.deepcopy(poses_cp)

    voxel_map = voxel_slam.frontend.VoxelFeatureMap(clouds, poses, voxel_size=voxel_size)
    voxel_feature_map = voxel_map.extract_voxel_features(
        ransac_distance_threshold=ransac_distance_threshold, 
        points_filter_function=filter_function
    )
    if enable_filter_voxels:
        voxel_map.filter_voxel_features(voxel_feature_map, 
                                        cosine_distance_threshold=filter_cosine_distance_threshold,
                                        preserve_non_informative_voxels=filter_preserve_non_informative_voxels)
    
    graph = voxel_slam.backend.FGraph(voxel_feature_map, number_of_poses=len(poses))
    optimized_poses = graph.get_optimized_poses()
    colored_pcd, color_to_voxel_id = voxel_map.get_colored_feature_clouds(voxel_feature_map, color_method="voxel")
    
    return colored_pcd, optimized_poses

In [22]:
clouds_path = "/home/ach/Desktop/datasets/vector_corridor/corridors_walk1.synced.lidar"
poses_path = "/home/ach/Desktop/datasets/vector_corridor/corridors_walk1.synced.gt.txt"

In [23]:
ts_multiplier = 1 / 1e9
ts_to_pcd_path = {float(x.stem) * ts_multiplier : x for x in pathlib.Path(clouds_path).iterdir()}
ts_to_quat = parse_trajectories_float(poses_path, ts_multiplier)

In [24]:
clouds, poses = read_vector_sequence(ts_to_quat, ts_to_pcd_path, start_of_sequence=80, number_of_clouds=100)

In [25]:
voxel_map = voxel_slam.VoxelFeatureMap(clouds, poses, voxel_size=2)

In [26]:
voxel_feature_map = voxel_map.extract_voxel_features(ransac_distance_threshold=0.01)

In [27]:
voxel_map.filter_voxel_features(voxel_feature_map)

In [28]:
colored_clouds, _ = voxel_map.get_colored_feature_clouds(voxel_feature_map, color_method="voxel")

Time (init): 0.013162624


In [29]:
optimized_poses = voxel_slam.FGraph(voxel_feature_map, len(poses)).get_optimized_poses()

FGraph initial error: 41.17875305817232
Chi2: 37.963707321639745


In [30]:
o3d.visualization.draw_geometries([voxel_slam.aggregate_map(colored_clouds, optimized_poses)])

#### Visualize clouds with GT poses

In [31]:
o3d.visualization.draw_geometries([voxel_slam.aggregate_map(clouds, poses, enable_color=True)])

In [55]:
class OptimizationConfig:
    def __init__(self) -> None:  
        self.window_size = 5
        self.voxel_size = 2
        self.ransac_distance_threshold = 0.005
        self.filter_function = lambda points: True
        self.enable_filter_voxels = True
        self.filter_cosine_distance_threshold = 0.2
        self.filter_preserve_non_informative_voxels = False

### Clustering optimization

In [56]:
def clustering_optimization(clouds, poses, submaps_config: OptimizationConfig, aggregated_map_config: OptimizationConfig):
    optimized_submaps = {}

    transformed_clouds = voxel_slam.VoxelFeatureMap(clouds, poses, voxel_size=submaps_config.voxel_size).transformed_clouds
 
    print("== Submaps ==")
    for i in range(0, len(clouds), submaps_config.window_size):
        optimized_submaps[i] = get_optimized_poses(
            clouds_cp=transformed_clouds[i:i+submaps_config.window_size],
            poses_cp=[np.eye(4) for _ in range(submaps_config.window_size)],
            voxel_size=submaps_config.voxel_size,
            ransac_distance_threshold=submaps_config.ransac_distance_threshold,
            filter_function=submaps_config.filter_function,
            enable_filter_voxels=submaps_config.enable_filter_voxels,
            filter_cosine_distance_threshold=submaps_config.filter_cosine_distance_threshold, 
            filter_preserve_non_informative_voxels=submaps_config.filter_preserve_non_informative_voxels
        )

    
    aggregated_submaps = {k: voxel_slam.aggregate_map(pcd, pose) for k, (pcd, pose) in optimized_submaps.items()}
    for pcd in aggregated_submaps.values():
        pcd.paint_uniform_color([0.0, 0.0, 0.0])

    o3d.visualization.draw_geometries(list(aggregated_submaps.values()))

    print("== Aggregated map ==")
    ahc_clouds, ahc_poses = get_optimized_poses(
        list(aggregated_submaps.values()), 
        [np.eye(4) for _ in range(len(aggregated_submaps))],
        voxel_size=aggregated_map_config.voxel_size,
        ransac_distance_threshold=aggregated_map_config.ransac_distance_threshold,
        filter_function=aggregated_map_config.filter_function,
        enable_filter_voxels=aggregated_map_config.enable_filter_voxels,
        filter_cosine_distance_threshold=aggregated_map_config.filter_cosine_distance_threshold,
        filter_preserve_non_informative_voxels=aggregated_map_config.filter_preserve_non_informative_voxels
    )

    return ahc_clouds, ahc_poses

In [57]:
submaps_config = OptimizationConfig()
aggregated_config = OptimizationConfig()

In [58]:
submaps_config.enable_filter_voxels = False 

In [ ]:
aggregated_clouds, aggregated_poses = clustering_optimization(clouds, poses, submaps_config, aggregated_config)

In [55]:
o3d.visualization.draw_geometries([voxel_slam.aggregate_map(aggregated_clouds, aggregated_poses)])